In [10]:
import os
import pandas as pd
from datetime import datetime
import math

def _get_list_of_humidities(df: pd.DataFrame):
	humidities = []
	columns = list(df.columns)
	for col in columns:
			current_line = (df[df[col] == '-']).reset_index(drop=True)
			if (len(current_line) != 0): 
				idx = columns.index(col) + 2
				if (idx > len(columns) - 1): continue
				humidities.append(float(current_line.iloc[0, idx]))
			else:
				current_line = (df[df[col].str.contains('\*') == True]).reset_index(drop=True)
				if (len(current_line) == 0): continue
				value = current_line.iloc[0, columns.index(col)]
				humidity = float(value.replace(' ', '')[9:])
				humidities.append(humidity)			

	return humidities

def _get_humidity_levels(df: pd.DataFrame):
	humidities = _get_list_of_humidities(df)
	h_len = len(humidities)
	if (h_len > 0):
		total = math.fsum(humidities)
		return round(total/h_len, 6)

def _get_time_in_seconds(time_object: datetime):
	return time_object.hour * 3600 + time_object.minute * 60 + time_object.second + time_object.microsecond / 1e6

def _convert_time_column_to_relative_seconds(df:pd.DataFrame, time_column_name='Time'):
	time_col = df[time_column_name].copy()
	for i, time in enumerate(time_col):
		try: time_obj = datetime.strptime(str(time), '%Y-%m-%d %H:%M:%S.%f')
		except:	time_obj = datetime.strptime(str(time), '%Y-%m-%d %H:%M:%S')
		time_col[i] = float(_get_time_in_seconds(time_obj))
	time_col = time_col - time_col.min()
	response = df.copy()
	response[time_column_name] = pd.to_numeric(time_col)
	return response

def _get_closest_line(target_in_seconds:float, df:pd.DataFrame, time_in_seconds_column_name='Time'):
	abs_diff = abs(df[time_in_seconds_column_name] - target_in_seconds)
	min_index = abs_diff.idxmin()
	return min_index

def diggest_files_into_single_dataframe(data_path:str):
	data_df = pd.DataFrame()
	final_df = pd.DataFrame()

	for filename in os.listdir(data_path):
		f = os.path.join(data_path, filename)
		if os.path.isfile(f):
			essay_number = f.split('\\')[1].split('E')[1][0:2]
			if (filename.startswith('E')):
				df = pd.read_excel(f)
				df = _convert_time_column_to_relative_seconds(df)
				data_df = df.copy()

			if (filename.startswith('U')):
				time = int(filename[5:filename.find('_min')]) * 60
				closest_line = _get_closest_line(time, data_df)

				df = pd.read_excel(f, dtype=str)
				humidity = _get_humidity_levels(df)

				line = dict(data_df.loc[closest_line])
				line['Umidade Produto [%]'] = humidity
				line['Ensaio'] = int(essay_number)

				final_df = pd.concat([final_df, pd.DataFrame([line])], ignore_index=True)

	return final_df

In [11]:
data_path = '../data/collected_data'

display(diggest_files_into_single_dataframe(data_path))

,Time,PT100 1 [ºC],PT100 2 [ºC],PT100 3 [ºC],PT100 4 [ºC],Temp. TH 1 [ºC],Umidade 1 [%],Vel. do Ar [m/s],Temp. TH 2 [ºC],Umidade 2 [%],Umidade Produto [%],Ensaio
0,0.000,27.143456,28.480464,1326.058821,1326.005777,29.006837,34.106580,1.075032,28.304104,70.879686,55.223333,0
1,898.795,26.381344,26.313029,-244.989090,-244.989090,30.783372,29.959138,1.075032,26.931477,89.835958,48.006667,0
2,1800.115,26.240221,26.156469,-244.989090,1326.005777,32.001662,26.997153,1.075032,26.642502,91.323760,41.003333,0
3,2698.281,34.239607,32.694989,1326.058821,-244.989090,32.156001,26.455326,1.075032,28.267982,67.753284,7.083333,0
4,3598.803,49.744406,48.653854,1326.058821,1326.005777,32.241379,25.844540,1.075032,44.782279,12.243013,2.283333,0
5,0.000,27.143456,28.480464,1326.058821,1326.005777,29.006837,34.106580,1.075032,28.304104,70.879686,61.353333,1
6,898.795,26.381344,26.313029,-244.989090,-244.989090,30.783372,29.959138,1.075032,26.931477,89.835958,51.775000,1
7,1800.115,26.240221,26.156469,-244.989090,1326.005777,32.001662,26.997153,1.075032,26.642502,91.323760,35.256667,1
8,2698.281,34.239607,32.694989,1326.058821,-244.989090,32.156001,26.455326,1.075032,28.267982,67.753284,4.863333,1
9,3598.803,49.744406,48.653854,1326.058821,1326.005777,32.241379,25.844540,1.075032,44.782279,12.243013,2.130000,1
